Aluno: Júlio Scholz

Como features, fiz uma análise de sentimentos nas reviews dos estabelecimentos, agrupei os pela média de cada estabelicimento e utilizei os seguintes classificadores:<br>
Knn, NaiveBayes, Decision Tree, Rede neural e Ada Boost <br> 

O melhor classificador foi o KNeighbors com 30 vizinhos, com um score de 81%, no kaggle ele resultou Public Score de 0.82. <br>
Utilizei as features: neg, pos, compound <br>
Neg é o quão negativa uma review é no intervalo [0 a 1] <br>
Pos é o quão positiva uma review é no intervalo [0 a 1] <br>
Compound é a pontuação agregada entre a positividade, negatividade e a neutralidade no intervalo [-1 a 1] <br>
<br>
Também testei um classificador de rede neural e o Ada Boost, ambos com resultados muito similares ao KNeighbors. <br>
Testei uma estratégia de votação por maioria utilizando a rede neural o Ada Boost e KNeighbors, e o score foi 0.01 menor que o KNeighbors sozinho.

In [211]:
import pandas as pd
import numpy as np

import string
import re

from collections import Counter
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.sentiment import SentimentIntensityAnalyzer

from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

In [212]:
#Carregando as bases de dado pra memória

revTrain = pd.read_csv("reviewsTrainToronto.csv")
revTrain.drop('user_id',axis=1,inplace=True)
revTrain.drop('date',axis=1,inplace=True)
revTrain

revTest = pd.read_csv("reviewsTestToronto.csv")
revTest.drop('user_id',axis=1,inplace=True)
revTest.drop('date',axis=1,inplace=True)
revTest

dfTrain = pd.read_csv("X_trainToronto.csv")
dfTest = pd.read_csv("X_testToronto.csv")

dfTrain['categories'] = dfTrain['categories'].astype(str)
dfTrain.set_index('business_id',inplace=True)
dfTest.set_index('business_id',inplace=True)


In [215]:
# Análise de sentimentos das reviews
# Aqui a ideia é gerar um novo dataframe com a estimativa de sentimento para cada review
# Esse novo dataframe será salvo em um novo arquivo csv para facilitar os testes, já que demorar mais de 10 minutos para rodar.

stop_words = stopwords.words('english')
stemmer = PorterStemmer()
stopwords_dict = Counter(stop_words)
PONTUACAO = string.punctuation

def remove_pontuacao(text):
    return text.translate(str.maketrans('', '', PONTUACAO))
def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stopwords_dict])
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])
def remove_url(text):
    return re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)

def preprocess(dfToProcess,filename):
    dfTexts = dfToProcess['text']
    dfTexts = dfTexts.str.lower()

    #Com o preprocessamento o score diminiui, optei por analisar o sentimento do texto puro
    
    """ dfTexts = dfTexts.apply(lambda text: remove_stopwords(text))
    dfTexts = dfTexts.apply(lambda text: remove_pontuacao(text))
    dfTexts= dfTexts.apply(lambda text: stem_words(text)) """
    
    sent = SentimentIntensityAnalyzer()
    dfTexts = pd.DataFrame(dfTexts)
    dfTexts.columns = ['text']
    dfTexts['result'] =  [sent.polarity_scores(b) for b in dfTexts['text']]

    # a função polarity_scores retorna um dicionário, ele será armazenado em uma coluna
    # e na sequência ele será tratado e dividido em 4 novas colunas no dataframe ('neg','neu','pos','compound')
    dfProcessed = pd.concat([dfToProcess.loc[:,['business_id']], pd.json_normalize(dfTexts['result']),dfTexts['text']],axis=1) 

    if filename:
        dfProcessed.to_csv(filename)
    return dfProcessed



In [214]:

#Gera o dataframe com a analaise de sentimento
""" dfTextTrain = preprocess(revTrain,'revTrainNotProcessada.csv')
dfTextTest = preprocess(revTest,'revTestNotProcessada') """

# carrega na memoria o arquivo gerado
dfTextTrain = pd.read_csv('revTrainNotProcessada.csv')
dfTextTest= pd.read_csv('revTestNotProcessada.csv')

try:
    dfTextTest.drop(['Unnamed: 0'],axis=1,inplace=True)
except:
    pass

try:
    dfTextTrain.drop(['Unnamed: 0'],axis=1,inplace=True)
except:
    pass

#Aqui os resultados da análise de sentimentos são agrupados por estabelicimento e pela média dos valores de cada estabelicimento
dfTextTrain = dfTextTrain.groupby('business_id').mean()
dfTextTest = dfTextTest.groupby('business_id').mean() 

dfTextTrain = pd.concat([dfTextTrain,dfTrain['destaque']],axis=1)
print(dfTextTrain.tail(1))
print(dfTextTest.tail(1))


                           neg       neu     pos  compound  destaque
business_id                                                         
zzvlwkcNR1CCqOPXwuvz2A  0.0475  0.820125  0.1325  0.477787         0
                             neg       neu       pos  compound
business_id                                                   
zzUj3ej4vm_DtvRxNvWDEw  0.053048  0.788563  0.158389  0.561529


In [276]:
#Utilizando o GridSearchCV para encontrar os melhos parametros para o KNN
X =  dfTextTrain.loc[:,['neg','pos','compound']]
y = dfTextTrain['destaque']

k_range=range(1, 100)
weight_options = ['uniform', 'distance']
param_grid = dict(n_neighbors=k_range, weights=weight_options)

grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='accuracy',n_jobs = -1).fit(X, y)
print(f'{grid.best_score_}')

print(grid.best_score_)
print(grid.best_params_)
print(grid.best_estimator_)

0.814872433105165
0.814872433105165
{'n_neighbors': 64, 'weights': 'distance'}
KNeighborsClassifier(n_neighbors=64, weights='distance')


In [265]:


X =  dfTextTrain.loc[:,['neg','pos','compound']]
y = dfTextTrain['destaque']

neigh = KNeighborsClassifier(n_neighbors=64, weights='distance')

scores = (cross_val_score(neigh, X, y, cv=5)).mean().round(5)
print(f'KNeighborsClassifier: {scores}')

naiveBayes = GaussianNB()
scores = (cross_val_score(naiveBayes, X, y, cv=5)).mean().round(5)
print(f'NaiveBayes: {scores}')

tree_model = tree.DecisionTreeClassifier()
modelo = tree_model.fit(X,y)
scores = (cross_val_score(modelo, X, y, cv=5)).mean().round(5)
print(f'DecisionTreeClassifier: {scores}')



KNeighborsClassifier: 0.81487
NaiveBayes: 0.76509
DecisionTreeClassifier: 0.74207


In [275]:

X =  dfTextTrain.loc[:,['neg','pos','compound']]
y = dfTextTrain['destaque']


ada = AdaBoostClassifier(n_estimators=100, random_state=22)
scores = (cross_val_score(ada, X, y, cv=5)).mean().round(5)
print(f'AdaBoostClassifier: {scores}') 

MLPC = MLPClassifier(solver='adam',alpha=1e-5,random_state=22)


scores = (cross_val_score(MLPC, X, y, cv=5)).mean().round(5)
print(f'MLPClassifier: {scores}')




AdaBoostClassifier: 0.8135
MLPClassifier: 0.81462


In [271]:

neighModel = neigh.fit(X, y)
resultado1 = neighModel.predict(dfTextTest.loc[:,['neg','pos','compound']])

modelMLPC = MLPC.fit(X, y)
resultado2 = modelMLPC.predict(dfTextTest.loc[:,['neg','pos','compound']])

ModelAda = ada.fit(X, y)
resultado3 = ModelAda.predict(dfTextTest.loc[:,['neg','pos','compound']])

res = pd.DataFrame({'res1':resultado1,'res2':resultado2,'res3':resultado3})
res

,res1,res2,res3
0,0,0,0
1,1,1,1
2,0,0,0
3,1,1,1
4,1,1,1
...,...,...,...
2831,0,0,0
2832,0,0,0
2833,0,0,0
2834,0,0,0


In [273]:

#sistema de votação por maioria gerando a previsão de destaque, soma todos os resultados se o for maior ou igual a dois a maioria "votou" por destaque, caso contrario nao tem destaque
dfTextTest['destaque'] = np.where(res['res1'] + res['res2'] + res['res3'] >= 2 , 1, 0)
dfTextTest['destaque'].to_csv('resultado.csv')
dfTextTest['destaque']

business_id
-03HVYxkeYWaafEpNJo1SA    0
-0M3o2uWBnQZwd3hmfEwuw    1
-0aOudcaAyac0VJbMX-L1g    0
-1NPtXJaDSoqxQPxt3rg3Q    1
-2TBP3ZGu7M-FmfoNJvbrQ    1
                         ..
zrQlLZLhToH4IHi8jCcbdQ    0
zwGu5H7MnilB9Fw7DZ7kXQ    0
zxU67OiGi8sF8zbdlTU0mQ    0
zzMcX99BPGSOFMZ4boINSQ    0
zzUj3ej4vm_DtvRxNvWDEw    0
Name: destaque, Length: 2836, dtype: int32

In [268]:
# Utilizando o KNN para prever o destaque do dataset de teste
neighModel = neigh.fit(X, y)
resultado = neighModel.predict(dfTextTest.loc[:,['neg','pos','compound']])

dfTextTest['destaque'] = resultado
dfTextTest['destaque'].to_csv('resultado.csv')
dfTextTest['destaque']

business_id
-03HVYxkeYWaafEpNJo1SA    0
-0M3o2uWBnQZwd3hmfEwuw    1
-0aOudcaAyac0VJbMX-L1g    0
-1NPtXJaDSoqxQPxt3rg3Q    1
-2TBP3ZGu7M-FmfoNJvbrQ    1
                         ..
zrQlLZLhToH4IHi8jCcbdQ    0
zwGu5H7MnilB9Fw7DZ7kXQ    0
zxU67OiGi8sF8zbdlTU0mQ    0
zzMcX99BPGSOFMZ4boINSQ    0
zzUj3ej4vm_DtvRxNvWDEw    0
Name: destaque, Length: 2836, dtype: int64